# MCP Standard Library
This notebook defines core components and utilities for the MCP framework,
including session initialization, tool management, and tool call functionality.

Chat-related elements are not included here (024_llms.ipynb)

In [ ]:
# |default_exp mcp_client

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2


In [ ]:
##### THE PLAN #####
# sessions = list/dict (initialize servers) 
# def mcp_agent_factory(chat,[mcp_sessions]):
# tools = session.get_tools()
# Converting tools to llm format
# return Diagram (Chat(promts, tool schema), Call_tools_func(session))


# call llm (Chat(promts, tool schema)
# def call tool(args, session):


In [ ]:
# | hide
from stringdale.core import get_git_root, load_env, checkLogs, json_render

In [ ]:
# | hide
load_env()

True

In [ ]:
# | export
import asyncio
import nest_asyncio
import json
import os
from typing import Tuple, List, Dict, Any, Optional
from contextlib import asynccontextmanager,AsyncExitStack

from fastmcp import Client, FastMCP
from fastmcp.client.transports import StreamableHttpTransport, PythonStdioTransport ,StdioTransport 


from anthropic import Anthropic

from openai import AsyncOpenAI

In [ ]:
nest_asyncio.apply()

In [ ]:
weather_path = str(get_git_root()/"stringdale/mcp_weather_server.py")
config = {
  "mcpServers": {
    "weather": {
      "command": "python",
      "args": [weather_path]
    }
  }
}


In [ ]:
mcp_client = Client(config)
async with mcp_client:
    mcp_tools = await mcp_client.list_tools()
# Note: client is still inside context and should be used within async with if you want to make calls
for i, tool in enumerate(mcp_tools):
    print(f"Tool {i+1}: {tool}")


Tool 1: name='get_alerts' title=None description='Get weather alerts for a US state.\n\n    Args:\n        state: Two-letter US state code (e.g. CA, NY)\n    ' inputSchema={'properties': {'state': {'title': 'State', 'type': 'string'}}, 'required': ['state'], 'title': 'get_alertsArguments', 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'get_alertsOutput', 'type': 'object'} icons=None annotations=None meta=None
Tool 2: name='get_forecast' title=None description='Get weather forecast for a location.\n\n    Args:\n        latitude: Latitude of the location\n        longitude: Longitude of the location\n    ' inputSchema={'properties': {'latitude': {'title': 'Latitude', 'type': 'number'}, 'longitude': {'title': 'Longitude', 'type': 'number'}}, 'required': ['latitude', 'longitude'], 'title': 'get_forecastArguments', 'type': 'object'} outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}

## Learning to work with FastMCP

In [ ]:
BRAVE_API_KEY = os.getenv("BRAVE_API_KEY")
#wiki_loc = str(get_git_root()/'sample_data/wiki')
wiki_loc = str(get_git_root()/'nbs/wiki')

In [ ]:
# Example usage of init_mcp_client_and_tools
config = {
  "mcpServers": {
    "filesystem": {
      "command": "npx",
      "args": [
        "-y",
        "@modelcontextprotocol/server-filesystem",
        wiki_loc,
      ]
    },
    "brave-search": {
            "command": "npx",
            "args": ["-y", "@modelcontextprotocol/server-brave-search"],
            "env": {
                "BRAVE_API_KEY": BRAVE_API_KEY
            }
        }
  }
}

mcp_client = Client(config)
async with mcp_client:
    mcp_tools = await mcp_client.list_tools()
# Note: client is still inside context and should be used within async with if you want to make calls
for i, tool in enumerate(mcp_tools):
    print(f"Tool {i+1}: {tool}")


Tool 1: name='filesystem_read_file' title='Read File (Deprecated)' description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.' inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']} outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False} icons=None annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None) meta={'_fastmcp': {'tags': []}}
Tool 2: name='filesystem_read_text_file' title='Read Text File' description="Read the complete contents of a file from the fi

Ok, now let's use one of the tools to make sure llm can work with them :)

First OpenAi

In [ ]:
client = AsyncOpenAI()

# allows async functions to run in jupyter notebook
nest_asyncio.apply()

# initialize the Gmail MCP client
gmail_mcp_client = mcp_client

tools = [{
"type": "function",
"function": {
    "name": tool.name,
    "description": tool.description,
    "parameters": tool.inputSchema
}
} for tool in mcp_tools]

user_input = "What events do I have on 28th of November of this year (2025)?"
# 1st LLM call to determine which tool to use
response = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": user_input}],
    tools=tools
)

In [ ]:
if response.choices[0].message.tool_calls:        
    tool_name = response.choices[0].message.tool_calls[0].function.name
    tool_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
    print(f"Tool Used: {tool_name}, Arguments: {tool_args}")

    # execute the tool called by the LLM
    async with gmail_mcp_client:
        tool_response = await gmail_mcp_client.call_tool(tool_name, tool_args)
        tool_response_text = tool_response.content[0].text    


Tool Used: google_calendar_find_events, Arguments: {'instructions': 'Find all events on 28th of November 2025', 'start_time': '2025-11-28T00:00:00', 'end_time': '2025-11-28T23:59:59'}


Now the same thing with Anthropic

In [ ]:
from anthropic import AsyncAnthropic

# Initialize Anthropic client (async)
client = AsyncAnthropic()

# allows async functions to run in jupyter notebook
nest_asyncio.apply()

# initialize the MCP client (same as before)
gmail_mcp_client = mcp_client

# Convert tools to Anthropic format
# Anthropic uses "input_schema" instead of "parameters"
tools = [{
    "name": tool.name,
    "description": tool.description,
    "input_schema": tool.inputSchema  # Note: "input_schema" not "parameters"
} for tool in mcp_tools]

user_input = "What events do I have on 28th of November of this year (2025)?"

# 1st LLM call to determine which tool to use
response = await client.messages.create(
    model="claude-3-haiku-20240307",  # changed to a cheaper option
    messages=[{"role": "user", "content": user_input}],
    tools=tools,
    max_tokens=1024
)


In [ ]:
response.content

[TextBlock(citations=None, text="Okay, let's use the ", type='text'),
 ToolUseBlock(id='toolu_01G7h3YrL42zT55wQuEhJpBr', input={'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}, name='google_calendar_find_events', type='tool_use')]

In [ ]:

# Extract tool use from response
tool_use_block = None
for block in response.content:
    if block.type == "tool_use":
        tool_use_block = block
        break

print(tool_use_block)

tool_name = tool_use_block.name
tool_args = tool_use_block.input  # Already a dict, no need to parse JSON
print(f"Tool Used: {tool_name}, Arguments: {tool_args}")

# Execute the tool called by the LLM
async with gmail_mcp_client:
    tool_response = await gmail_mcp_client.call_tool(tool_name, tool_args)
    tool_response_text = tool_response.content[0].text


ToolUseBlock(id='toolu_01G7h3YrL42zT55wQuEhJpBr', input={'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}, name='google_calendar_find_events', type='tool_use')
Tool Used: google_calendar_find_events, Arguments: {'instructions': 'Find events on November 28, 2025.', 'start_time': '2025-11-28T23:59:00', 'end_time': '2025-11-28T00:00:00'}


In [ ]:
tool_response_text

'{"results":[{"kind":"calendar#event","etag":"\\"3478771113680000\\"","id":"0o8hmc4bn6uqn1h58gafeqqmog_20251128","status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver","created":"2024-12-06T02:23:40.000Z","updated":"2025-02-12T18:39:16.840Z","summary":"Дима Зицер\'s birthday","creator":{"email":"olga.a.soldatenko@gmail.com","self":true},"organizer":{"email":"olga.a.soldatenko@gmail.com","self":true},"start":{"date":"2025-11-28","date_pretty":"Nov 28, 2025","dateTime_pretty":"Nov 28, 2025 12:00AM","dateTime":"2025-11-28","time":""},"end":{"date":"2025-11-29","date_pretty":"Nov 29, 2025","dateTime_pretty":"Nov 29, 2025 12:00AM","dateTime":"2025-11-29","time":""},"recurringEventId":"0o8hmc4bn6uqn1h58gafeqqmog","originalStartTime":{"date":"2025-11-28"},"transparency":"transparent","visibility":"private","iCalUID":"0o8hmc4bn6uqn1h58gafeqqmog@google.com","sequence":0,"reminde

In [ ]:
tool_name

'google_calendar_find_events'

In [ ]:
tool_args

{'instructions': 'Find events on November 28, 2025.',
 'start_time': '2025-11-28T23:59:00',
 'end_time': '2025-11-28T00:00:00'}

In [ ]:
tool_response_text

'{"results":[{"kind":"calendar#event","etag":"\\"3478771113680000\\"","id":"0o8hmc4bn6uqn1h58gafeqqmog_20251128","status":"confirmed","htmlLink":"https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver","created":"2024-12-06T02:23:40.000Z","updated":"2025-02-12T18:39:16.840Z","summary":"Дима Зицер\'s birthday","creator":{"email":"olga.a.soldatenko@gmail.com","self":true},"organizer":{"email":"olga.a.soldatenko@gmail.com","self":true},"start":{"date":"2025-11-28","date_pretty":"Nov 28, 2025","dateTime_pretty":"Nov 28, 2025 12:00AM","dateTime":"2025-11-28","time":""},"end":{"date":"2025-11-29","date_pretty":"Nov 29, 2025","dateTime_pretty":"Nov 29, 2025 12:00AM","dateTime":"2025-11-29","time":""},"recurringEventId":"0o8hmc4bn6uqn1h58gafeqqmog","originalStartTime":{"date":"2025-11-28"},"transparency":"transparent","visibility":"private","iCalUID":"0o8hmc4bn6uqn1h58gafeqqmog@google.com","sequence":0,"reminde

In [ ]:
#It's for openai 
client = AsyncOpenAI()
# 2nd LLM call to determine final response
res = await client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": user_input},
        {"role": "function", "name": tool_name, "content": tool_response_text},
    ]        
)

response = res.choices[0].message.content
response


"On November 28, 2025, you have the following events scheduled:\n\n1. **Dima Зицер's Birthday**\n   - **Time:** All day event (24 hours starting from November 28, 2025)\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=MG84aG1jNGJuNnVxbjFoNThnYWZlcXFtb2dfMjAyNTExMjggb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Vancouver)\n\n2. **Zoom Mom**\n   - **Time:** November 28, 2025, from 9:00 AM to 10:00 AM\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=MnVyc2I5aTcwcjBsMWEzbzlhYm02M2JzdHAgb2xnYS5hLnNvbGRhdGVua29AbQ&ctz=America/Los_Angeles)\n   - **Hangout Link:** [meet.google.com/zhm-qhhq-zyc](https://meet.google.com/zhm-qhhq-zyc)\n\n3. **Dean Eats at Uni**\n   - **Time:** November 28, 2025, from 1:00 PM to 2:00 PM\n   - **Link:** [View Event](https://www.google.com/calendar/event?eid=ZzV0aDFsNXNwY3IwZGZrNTE1dnJrc3M1bzRfMjAyNTExMjhUMjEwMDAwWiBvbGdhLmEuc29sZGF0ZW5rb0Bt&ctz=America/Vancouver)\n\nMake sure to check your calendar for any updates or additional 

# Execute mcp tool factory

In [ ]:
#| export
def mcp_tool_executor_factory(mcp_client):
    """
    Factory function that creates an execute_mcp_tool function bound to a specific mcp_client.
    
    Args:
        mcp_client: The MCP client instance to use for tool execution
        
    Returns:
        An async function that executes tools using the provided client
    """
    async def execute_mcp_tool(tool):
        """
        Extracts the tool name and arguments from the tool dict,
        executes the tool via mcp_client, and returns both:
          1. the OpenAI tool_result-style message, and 
          2. the raw tool_result itself.
        """
        tool_name = tool['name']
        tool_args = tool['input']
        tool_id = tool['id']
        async with mcp_client:
            tool_result = await mcp_client.call_tool(tool_name, tool_args)
        # Extract text from content objects to make it serializable
        content_texts = [item.text for item in tool_result.content if hasattr(item, 'text')]
        return {
            "tool_name": tool_name,
            "tool_args": tool_args,
            "tool_id": tool_id,
            "tool_content_texts": content_texts,
        }
    return execute_mcp_tool

In [ ]:
exc = mcp_tool_executor_factory(mcp_client)

In [ ]:
tool = {'name': 'filesystem_search_files', 'input': {'path': 'wiki', 'pattern': '*dogs*'}, 'id': 'call_rnVqcB8nKciCL4NOkMFsRBMC'}
a = await exc(tool)

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()